# The Revenge of Rocchio's Angels

We will be #1 this time

Local Script Dependencies

In [ ]:
!python -c "import platform; print(platform.platform()); import sys; print(sys.version)"

In [11]:
from engine import SearchEngine
from evaluate_map import *

In [12]:
se = SearchEngine()

In [13]:
se.set_searcher()

In [ ]:
results = se.get_top_k("international organized crime", k=50)

In [ ]:
results[0][0][:300]

In [14]:
topics = load_topics("Data/queriesROBUST.txt")

In [15]:
# This one creates a file called "run.txt" with submitting format, can change file name
se.search_all_queries(topics, 500)

In [16]:
qrels = load_qrels("Data/qrels_50_Queries")   # or "qrel301.txt"
run   = load_run("run.txt")

map_score, ap_by_q = mean_average_precision(qrels, run)
map_score

0.24225420721175706